In [1]:
import pandas as pd
from pathlib import Path

df = pd.read_csv("../data/staging/matches_2018_raw.csv")
df.shape

(64, 10)

In [3]:
# Table de référence des équipes (2018)
teams_ref_2018 = pd.Series(
    pd.concat([df["home_team"], df["away_team"]], ignore_index=True).unique()
).sort_values().reset_index(drop=True).to_frame(name="team")

teams_ref_2018.head(), teams_ref_2018.shape

(        team
 0  Argentina
 1  Australia
 2    Belgium
 3     Brazil
 4   Colombia,
 (32, 1))

In [4]:
# Export de la liste unique des équipes, pour alimenter le dictionnaire commun
Path("../data/processed").mkdir(parents=True, exist_ok=True)

teams_ref_2018.to_csv("../data/processed/teams_ref_2018.csv", index=False)

print("✅ Export OK -> ../data/processed/teams_ref_2018.csv")

✅ Export OK -> ../data/processed/teams_ref_2018.csv


In [5]:
teams_ref_2018.shape
teams_ref_2018["team"].head(10)

0     Argentina
1     Australia
2       Belgium
3        Brazil
4      Colombia
5    Costa Rica
6       Croatia
7       Denmark
8         Egypt
9       England
Name: team, dtype: object